In [78]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [43]:
# 读取预处理之前的数据
data = pd.read_csv('temp/因子数据初提取.csv',encoding = 'GBK',engine = 'python')

In [44]:
# 读取需要预测的数据
data_7 = data[data['年份（年末）'] == 7]

fac_col = ['总股本',
         '每股资本公积',
         '每股未分配利润',
         '基本每股收益',
         '基本每股收益同比增长(%)',
         '每股净资产',
         '每股现金流',
         '每股营业收入',
         '摊薄净资产收益率同比增长(%)',
         '净利润同比增长',
         '上市年限',
         '交通运输、仓储和邮政业',
         '住宿和餐饮业',
         '信息传输、软件和信息技术服务业',
         '制造业',
         '房地产业',
         '批发和零售业',
         '电力、热力、燃气及水生产和供应业',
         '综合',
         '采矿业',
         '是否有行业龙头概念',
         '股价（年末最后20个交易日收盘价加权平均）',
         '股价年化波动率',
         '本年送转比例',
         '近两年送转比例平均']

# 提取第七年单因素方差分析筛选过的因子
data_pred = data_7[['股票编号','年份（年末）']].copy()
data_pred[fac_col] = data_7[fac_col]

# 重置索引
data_pred.reset_index(inplace = True)
data_pred.drop(axis = 0,columns = 'index',inplace = True)

In [45]:
data_pred.shape

(3466, 27)

In [46]:
len(data_pred.columns[2:])

25

In [47]:
data_pred.isnull().sum()

股票编号                      0
年份（年末）                    0
总股本                       0
每股资本公积                   10
每股未分配利润                   1
基本每股收益                    1
基本每股收益同比增长(%)             1
每股净资产                    25
每股现金流                     0
每股营业收入                    0
摊薄净资产收益率同比增长(%)          52
净利润同比增长                   0
上市年限                      0
交通运输、仓储和邮政业               0
住宿和餐饮业                    0
信息传输、软件和信息技术服务业           0
制造业                       0
房地产业                      0
批发和零售业                    0
电力、热力、燃气及水生产和供应业          0
综合                        0
采矿业                       0
是否有行业龙头概念                 0
股价（年末最后20个交易日收盘价加权平均）     0
股价年化波动率                   0
本年送转比例                    0
近两年送转比例平均                 0
dtype: int64

In [48]:
# 缺失值处理
for i in data_pred.columns[2:]:
    for j in range(len(data_pred)):
        if (data_pred[i].isnull())[j]: # 若为空值，则用该年份该因子的平均值填充
            year = data_pred.iloc[j]['年份（年末）']
            avg = np.mean(data_pred[data_pred['年份（年末）'] == year][i])
            data_pred[i][j] = avg

In [49]:
data_pred.isnull().sum()

股票编号                     0
年份（年末）                   0
总股本                      0
每股资本公积                   0
每股未分配利润                  0
基本每股收益                   0
基本每股收益同比增长(%)            0
每股净资产                    0
每股现金流                    0
每股营业收入                   0
摊薄净资产收益率同比增长(%)          0
净利润同比增长                  0
上市年限                     0
交通运输、仓储和邮政业              0
住宿和餐饮业                   0
信息传输、软件和信息技术服务业          0
制造业                      0
房地产业                     0
批发和零售业                   0
电力、热力、燃气及水生产和供应业         0
综合                       0
采矿业                      0
是否有行业龙头概念                0
股价（年末最后20个交易日收盘价加权平均）    0
股价年化波动率                  0
本年送转比例                   0
近两年送转比例平均                0
dtype: int64

In [50]:
data_pred.columns[3:11]

Index(['每股资本公积', '每股未分配利润', '基本每股收益', '基本每股收益同比增长(%)', '每股净资产', '每股现金流',
       '每股营业收入', '摊薄净资产收益率同比增长(%)'],
      dtype='object')

In [51]:
# 极端值处理
for i in data_pred.columns[3:11]:
    mean = data_pred.describe()[i]['mean']
    std = data_pred.describe()[i]['std']
    #print(i)
    #print('mean:%.2f'%mean)
    #print('std:%.2f'%std)
    for j in range(len(data_pred)):
        if data_pred[i][j] > mean + 3*std:
            data_pred[i][j] = mean + 3*std
        elif data_pred[i][j] < mean - 3*std:
            data_pred[i][j] = mean - 3*std

In [52]:
data_pred.describe()

,股票编号,年份（年末）,总股本,每股资本公积,每股未分配利润,基本每股收益,基本每股收益同比增长(%),每股净资产,每股现金流,每股营业收入,...,房地产业,批发和零售业,电力、热力、燃气及水生产和供应业,综合,采矿业,是否有行业龙头概念,股价（年末最后20个交易日收盘价加权平均）,股价年化波动率,本年送转比例,近两年送转比例平均
count,3466.000000,3466.0,3.466000e+03,3466.000000,3466.000000,3466.000000,3466.000000,3466.000000,3466.000000,3466.000000,...,3466.000000,3466.000000,3466.000000,3466.000000,3466.000000,3466.000000,3466.000000,3466.000000,3466.000000,3466.000000
mean,1733.500000,7.0,1.740856e+09,2.117344,1.470473,0.386744,4.550138,4.858675,0.157358,5.978280,...,0.036065,0.047317,0.030871,0.006924,0.021639,0.122620,14.790861,0.348058,0.225274,0.217875
std,1000.692344,0.0,1.152613e+10,1.857322,1.668506,0.498584,278.326589,3.034256,0.988254,7.043480,...,0.186478,0.212346,0.172994,0.082936,0.145522,0.328048,16.664863,0.121286,0.378512,0.318029
min,1.000000,7.0,4.216000e+07,-1.184800,-4.864241,-1.697218,-1913.611835,-2.658300,-3.712726,0.019200,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.169499,0.000000,0.000000,0.000000
25%,867.250000,7.0,2.725216e+08,0.787450,0.588625,0.119400,-32.336050,2.776550,-0.201300,1.980825,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.341860,0.270109,0.015000,0.025000
50%,1733.500000,7.0,5.546795e+08,1.658600,1.192350,0.298950,2.234700,4.232000,0.041400,3.693050,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.139625,0.331301,0.081500,0.100000
75%,2599.750000,7.0,1.141754e+09,2.869000,2.102675,0.582800,40.000000,6.137625,0.412875,7.147425,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17.773328,0.410100,0.281500,0.324553
max,3466.000000,7.0,3.564070e+11,9.655978,7.902542,2.483676,1908.629125,16.467529,4.051351,44.179662,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,647.769898,1.500275,10.999000,8.893000


导入模型，预测在次年（第八年）高送转的情况

In [93]:
import pickle
from xgboost import XGBClassifier
from sklearn.externals import joblib
file_name = 'Model/XGBoost.dat'
model = pickle.load(open(file_name, "rb"))

In [94]:
predictions = model.predict(data_pred.iloc[:,2:])

In [96]:
print('预测将在第八年高送转的股票数为:%d'%np.sum(predictions))

预测将在第八年高送转的股票数为:223


In [63]:
trans_list = data_pred[predictions == 1]['股票编号'].values

In [67]:
print(trans_list)

[  10   23   40   41   50   60   88  165  179  187  188  191  194  215
  219  224  258  265  266  293  299  343  352  356  373  387  398  402
  403  415  417  458  468  470  493  509  518  534  537  570  598  603
  621  637  700  742  747  748  749  762  764  789  793  795  842  864
  874  902  915  922  955  957  968 1003 1013 1026 1033 1044 1071 1095
 1101 1122 1144 1161 1166 1182 1183 1252 1260 1313 1317 1321 1332 1341
 1346 1366 1386 1389 1395 1450 1460 1481 1483 1499 1510 1524 1535 1543
 1548 1559 1581 1590 1607 1609 1616 1646 1660 1668 1694 1754 1759 1765
 1769 1785 1812 1815 1817 1835 1860 1864 1874 1886 1909 1913 1922 1933
 1943 1967 1985 1998 2024 2028 2034 2038 2040 2079 2086 2118 2119 2120
 2124 2152 2154 2197 2216 2233 2247 2279 2307 2340 2359 2386 2390 2409
 2437 2438 2460 2480 2494 2520 2533 2550 2552 2559 2560 2599 2601 2630
 2644 2652 2673 2686 2687 2697 2703 2710 2724 2759 2774 2852 2881 2905
 2924 2949 2958 2971 2977 2991 3026 3042 3044 3056 3076 3078 3091 3093
 3113 

In [106]:
pred_save = pd.DataFrame()
pred_save['股票编号'] = data_pred['股票编号']
pred_save['预测第八年是否高送转'] = predictions
pred_save.to_csv('temp/预测结果.csv',index = False)